<a href="https://colab.research.google.com/github/vishnubanna/stocktrader/blob/master/recurrent_nural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import random
from collections import deque
from sklearn import preprocessing
import datetime as dt

import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

In [0]:
def getData(ticker, number_of_days = 10000, days_ago = 0):
    try:
        print(ticker)
        start = dt.datetime.today() - dt.timedelta(days = number_of_days)
        end = dt.datetime.today() - dt.timedelta(days = days_ago)
        df = pdr.get_data_yahoo(ticker, start, end)
        print(df.tail())
        return df
    except:
        pass
    
def dataSpecify(df):
    df['e50MAvg'] = df['Close'].ewm(span = 50, adjust = False, min_periods = 0).mean()
    df['e26MAvg'] = df['Close'].ewm(span = 26, adjust = False, min_periods = 0).mean()
    df['e12MAvg'] = df['Close'].ewm(span = 12, adjust = False, min_periods = 0).mean()

    df['MACD'] = df['e12MAvg'] - df['e26MAvg']

    df['MFlowMult'] = ((df.Close - df.Low) - (df.High - df.Close))/(df.High - df.Low)
    df['MFlowVol'] = (df['Volume'])*df['MFlowMult']
    df['ADL'] = df['MFlowVol'].values.cumsum()
    #df['ADL'] = df['ADL'].shift(1)
    df.fillna(0, inplace = True)
    df['e50AAvg'] = df['ADL'].ewm(span = 50, adjust = False, min_periods = 0).mean()
    df['e10MAvg'] = df['ADL'].ewm(span = 10, adjust = False, min_periods = 0).mean()
    df['e3MAvg'] = df['ADL'].ewm(span = 3, adjust = False, min_periods = 0).mean()

    df['Chakin'] = df['e3MAvg'] - df['e10MAvg']
    
    #df = df.drop(['e26MAvg', 'e12MAvg', 'High', 'Low', 'Adj Close', 'e3MAvg', 'e10MAvg'], axis = 1)
    df = df[['Open', 'Close','e26MAvg' , 'e50MAvg', 'MACD', 'ADL', 'Chakin', 'High', 'Low']]
    print(df.head(), df.tail())
    return df

df = getData('AAPL')
df = dataSpecify(df)

In [0]:
SEQ_LEN = 60 #how many sequential data points correspond to one prediction. in out case we are using the last 60 points to to make a prediction. so the last 60 days including today can be used to predict tommorow
FUTURE_PERIOD_PREDICT = 10 # how many days in advance do you want to predict
DATA_TO_PREDICT = 'e50MAvg' # what do you want to predict 

def classify(current_value, future_value):
    if float(current_value) <= float(future_value): # rather than predict the future price, if the furture is better or higher, get 1 or hold
        return 1
    else: # else 0 sell
        return 0
    
    
def process_df(df, scale = 0):
    #scales = []
    #scale = 0
    # normalize the colomns: all have different magnitude of values, this reduces the magnitude down by using percent change from point to point, reduces so all numbers in df are within the same range or a similar range
    for col in df.columns: 
        # don't change the target colomn
        if col != 'targets':
            #all have different magnitude of values, this reduces the magnitude down by using percent change from point to point, reduces so all numbers in df are within the same range or a similar range
            df[col] = df[col].pct_change()
            df[col].replace([np.inf, -np.inf], np.nan, inplace = True)
            #df.dropna(inplace = True)
    
    df.dropna(inplace = True)
#     for col in df.columns:
#         if col != 'targets':
#             df[col] = preprocessing.scale(df[col].values)
    target = np.asarray([df['targets'].values])
    #print(target.shape)
    
    #print(df.tail())
    df = df.drop(['targets'], axis = 1)
    
    if scale == 0:
        scale = preprocessing.RobustScaler().fit(df.values)
        scaled_df = scale.transform(df.values)
    else:
        scaled_df = scale.transform(df.values)
    #print(scaled_df.shape)   
    scaled_df = np.concatenate((scaled_df, target.T), axis = 1)
    #print(scaled_df)
  
    sequential_data = [] # our data points will go here 
    prev_days = deque(maxlen = SEQ_LEN) #so it compiles 60 days of data into one list, so on the day today it puts the past 60 days including today, for yesterday it dos the past 60 day unil yesterday including yerstery, it stops
        # when it cal no longer populate a sinle list with 60 days of data 
 
    for i in scaled_df:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == 60: 
            sequential_data.append([np.array(prev_days), i[-1]])
       
    print((sequential_data[0]))
    print((sequential_data[len(sequential_data)-1]))
    
    #shuffle the sequenced arrays;; to prevent the model from memeorizing the data set, and thinking pushing the training set and testing set to not represent each other 
    random.shuffle(sequential_data)
    
    #balence the data set:: make sure therea re the ame number of buys and sells
    buys = []
    sells = []
    
    for data, target in sequential_data:
        if target == 1:
            buys.append([data, target])
        else:
            sells.append([data, target])
    
    random.shuffle(buys)
    random.shuffle(sells)
    
    
    size = min(len(buys), len(sells))
    
    
    buys = buys[:size]
    sells = sells[:size]
    
    print(len(buys), len(sells))
    
    fullset = buys + sells
    random.shuffle(fullset)
    
    X = []
    Y = []
    
    for data, target in fullset:
        X.append(data)
        Y.append(target)
                
    return np.array(X), np.array(Y), scale


def process_df2(df, scale = 0):
    #scales = []
    #scale = 0
    # normalize the colomns: all have different magnitude of values, this reduces the magnitude down by using percent change from point to point, reduces so all numbers in df are within the same range or a similar range
    for col in df.columns: 
        # don't change the target colomn
        if col != 'targets':
            #all have different magnitude of values, this reduces the magnitude down by using percent change from point to point, reduces so all numbers in df are within the same range or a similar range
            df[col] = df[col].pct_change()
            df[col].replace([np.inf, -np.inf], np.nan, inplace = True)
            #df.dropna(inplace = True)
    
    df.dropna(inplace = True)
#     for col in df.columns:
#         if col != 'targets':
#             df[col] = preprocessing.scale(df[col].values)
    target = np.asarray([df['targets'].values])
    #print(target.shape)
    
    #print(df.tail())
    df = df.drop(['targets'], axis = 1)
    
    if scale == 0:
        scale = preprocessing.RobustScaler().fit(df.values)
        scaled_df = scale.transform(df.values)
    else:
        scaled_df = scale.transform(df.values)
    #print(scaled_df.shape)   
    scaled_df = np.concatenate((scaled_df, target.T), axis = 1)
    #print(scaled_df)
  
    sequential_data = [] # our data points will go here 
    prev_days = deque(maxlen = SEQ_LEN) #so it compiles 60 days of data into one list, so on the day today it puts the past 60 days including today, for yesterday it dos the past 60 day unil yesterday including yerstery, it stops
        # when it cal no longer populate a sinle list with 60 days of data 
 
    for i in scaled_df:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == 60: 
            sequential_data.append([np.array(prev_days), i[-1]])
       
    #print((sequential_data))
    
    #shuffle the sequenced arrays;; to prevent the model from memeorizing the data set, and thinking pushing the training set and testing set to not represent each other 
    #random.shuffle(sequential_data)
    
    #balence the data set:: make sure therea re the ame number of buys and sells
    buys = []
    sells = []
    
    for data, target in sequential_data:
        if target == 1:
            buys.append([data, target])
        else:
            sells.append([data, target])
    
    #random.shuffle(buys)
    #random.shuffle(sells)
    
    
    #size = min(len(buys), len(sells))
    
    
    #buys = buys[:size]
    #sells = sells[:size]
    
    print(len(buys), len(sells))
    
    fullset = buys + sells
    random.shuffle(fullset)
    
    X = []
    Y = []
    
    for data, target in sequential_data:
        X.append(data)
        Y.append(target)
                
    return np.array(X), np.array(Y), scale

            
    

df['future'] = df[DATA_TO_PREDICT].shift(-FUTURE_PERIOD_PREDICT)
df['targets'] = list(map(classify, df[DATA_TO_PREDICT], df['future']))
df.dropna(inplace = True)
df = df.drop(['future'], axis = 1)
#df = df.drop(['0'], axis = 1) #figure out how
print(df.head())
print(df.tail())


#split the training data and the testing data:: we ahve to sort and take a chunk because if we randomized data(if it sequential), the data will have a very similar counterpart of itself inside the training set. 
#at which point it will, have high accuracy even if it not truly that accurate
# with sequencial data only

# they should be in order, but just make sure
dates = sorted(df.index.values)
last_5pct_data = dates[-int(0.05*len(dates))]
print(last_5pct_data)

train_df = df[(df.index < last_5pct_data)] # all data point with data less than the last 5pct data = training, all data grater = first 5 pct
test_df = df[(df.index >= last_5pct_data)]

print(train_df.tail())

#process_df(train_df)
x_train, y_train, scale = process_df(train_df)
x_test, y_test, scale2 = process_df(test_df, scale = scale) 
x_test1, y_test1, scale3 = process_df2(test_df, scale = scale)



#print(x_test, y_test)

    

In [0]:
pip install tensorflow==2.0.0-alpha0

In [0]:
import tensorflow as tf
import tensorflow.keras as ks
from tensorflow.keras.layers import Dense, LSTM, Dropout, InputLayer, BatchNormalization

layers = [
    InputLayer(input_shape = (x_train.shape[1:] )),
    LSTM(units = 50, activation = 'tanh', return_sequences = True), 
    Dropout(0.2),
    BatchNormalization(),
    LSTM(units = 50, activation = 'tanh', return_sequences = True), 
    Dropout(0.2),
    BatchNormalization(),
    LSTM(units = 50, activation = 'tanh'), 
    Dropout(0.2),
    BatchNormalization(),
    Dense(units = 32, activation = 'relu'), 
    Dropout(0.2),
    Dense(units = 2, activation = 'softmax'), 
    
]
model = ks.Sequential(layers)

opt = ks.optimizers.Adam(lr = 0.001, decay = 0.000001)

model.compile(loss = ks.losses.SparseCategoricalCrossentropy(), 
             optimizer = opt, 
             metrics = ['accuracy'])

model.fit(x_train, y_train, batch_size = 30, epochs = 7, validation_data = (x_test, y_test))

model.predict(x_test, verbose = 0)

In [0]:
test = model.predict(x_test1, verbose = 0)
print(y_test1)
test = test.tolist()

decision = ['buy' if (max(tes[0],tes[1]) == tes[1]) else 'sell' for tes in test]
test = ['buy' if y == 1 else 'sell' for y in y_test1]

print(decision)
print(test)

                Open     Close   e50MAvg      MACD       ADL    Chakin  targets
Date                                                                           
2019-04-25 -0.002556 -0.009075  0.003351 -0.026472 -0.000771 -0.364350        0
2019-04-26 -0.009331 -0.004774  0.003006 -0.046782  0.000454 -0.146574        1
2019-04-29 -0.002440  0.001517  0.002944 -0.045174 -0.000304 -0.301522        1
2019-04-30 -0.006556 -0.019256  0.002011 -0.110670 -0.000601 -0.664775        1
2019-05-01  0.033586  0.049086  0.003947  0.065975 -0.001767 -4.386830        0